## Import

In [59]:
import json
import folium 
import requests
import numpy as np
import regex as re
import pandas as pd
import pickle as pkl
import shapely as shp
import geopandas as gpd
from copy import deepcopy
from sodapy import Socrata
import plotly.express as px
from pyomo.environ import *
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Data Reading

In [2]:
food_data_path = r"C:\Users\Marco\Documents\GitHub\commercial-activities-Milan\Data\02_Processed\food_data.pkl"
food_data = pkl.load(open(food_data_path, 'rb'))

In [3]:
building_data_path = r"C:\Users\Marco\Documents\GitHub\commercial-activities-Milan\Data\02_Processed\building_data.pkl"
building_data = pkl.load(open(building_data_path, 'rb'))

In [4]:
NIL_data_path = r"C:\Users\Marco\Documents\GitHub\commercial-activities-Milan\Data\02_Processed\NIL_data.pkl"
NIL_data = pkl.load(open(NIL_data_path, 'rb'))

In [6]:
distances_matrix_path = r"C:\Users\Marco\Documents\GitHub\commercial-activities-Milan\Data\02_Processed\distances_matrix2.pkl"
distances_matrix = pkl.load(open(distances_matrix_path, 'rb'))

In [9]:
distances_matrix_index_path = r"C:\Users\Marco\Documents\GitHub\commercial-activities-Milan\Data\02_Processed\distances_matrix2_index.pkl"
distances_matrix_index = pkl.load(open(distances_matrix_index_path, 'rb'))

In [10]:
distances_matrix_index

building_index                                         food_index   
0             529  [3, 44, 46, 56, 60, 63, 114, 130, 156, 162, 19...  \
1           21093  [13, 63, 86, 97, 109, 110, 114, 116, 120, 121,...   
2             935  [3, 44, 46, 56, 60, 63, 114, 130, 156, 162, 19...   
3             659  [3, 14, 22, 23, 24, 44, 58, 60, 63, 114, 118, ...   
4           11606  [26, 34, 74, 75, 103, 106, 111, 113, 122, 153,...   
..            ...                                                ...   
83          30026  [30, 31, 55, 66, 117, 131, 133, 163, 164, 204,...   
84          18883  [9, 53, 69, 71, 193, 221, 295, 393, 394, 395, ...   
85          28161  [1, 21, 27, 57, 78, 85, 93, 94, 101, 170, 223,...   
86          19691  [12, 59, 297, 298, 333, 334, 419, 543, 565, 58...   
87           4785  [105, 169, 195, 196, 252, 261, 262, 263, 264, ...   

                                        food_shop_tag   
0   [supermarket, convenience, supermarket, superm...  \
1   [supermarket, supermarket, supermarket, superm...   
2   [supermarket, convenience, supermarket, superm...   
3   [supermarket, supermarket, supermarket, superm...   
4   [supermarket, supermarket, supermarket, superm...   
..                                                ...   
83  [bakery, supermarket, convenience, convenience...   
84  [convenience, bakery, convenience, supermarket...   
85  [supermarket, bakery, bakery, convenience, sup...   
86  [bakery, supermarket, convenience, supermarket...   
87  [supermarket, convenience, bakery, wine, baker...   

                                 building_location   
0                    [POINT (9.1165892 45.520052)]  \
1                    [POINT (9.158085 45.5266971)]   
2           [POINT (9.1147411 45.513653000000005)]   
3                   [POINT (9.1363546 45.4843686)]   
4           [POINT (9.2269889 45.452410900000004)]   
..                                             ...   
83            [POINT (9.174584300000001 45.45552)]   
84          [POINT (9.1654395 45.423555400000005)]   
85          [POINT (9.127125900000001 45.4299655)]   
86                  [POINT (9.1635525 45.4697339)]   
87  [POINT (9.187675200000001 45.480990600000005)]   

                                   food_shop_location  
0   [POINT (9.1454007 45.4901442), POINT (9.152746...  
1   [POINT (9.1653078 45.5004259), POINT (9.137551...  
2   [POINT (9.1454007 45.4901442), POINT (9.152746...  
3   [POINT (9.1454007 45.4901442), POINT (9.154999...  
4   [POINT (9.226826200000001 45.462338900000006),...  
..                                                ...  
83  [POINT (9.1712789 45.455773), POINT (9.1752873...  
84  [POINT (9.1626739 45.431279100000005), POINT (...  
85  [POINT (9.1420016 45.447585000000004), POINT (...  
86  [POINT (9.159693 45.466715900000004), POINT (9...  
87  [POINT (9.193008800000001 45.4803834), POINT (...  

[88 rows x 5 columns]

In [55]:
fig = go.Figure()

for geom in NIL_data.geometry:
    geom = geom.exterior.coords.xy
    fig.add_trace(go.Scattermapbox(
        lat=list(geom[1]),
        lon=list(geom[0]),
        mode='lines',
        marker=dict(
            color='black',
            size=0.5,
        ),
        showlegend=False,
        hoverinfo='skip',
        fill='toself',
        fillcolor="rgba(0,0,0,0.1)"
    ))


lat = [p[0].y if type(p[0])==shp.geometry.point.Point else None for p in distances_matrix_index.building_location]
lon = [p[0].x if type(p[0])==shp.geometry.point.Point else None for p in distances_matrix_index.building_location]

fig.add_trace(go.Scattermapbox(
        lat=lat,
        lon=lon,
        mode='markers',
        marker=dict(
          color=["blue" for x in lat],
          size=6,
        ), name="buildings"))


lat_list = [p if len(p)>0 else None for p in distances_matrix_index.food_shop_location]
lon_list = [p if len(p)>0 else None for p in distances_matrix_index.food_shop_location]

lat = []
lon = []

for i in range(len(lat_list)):
  if lat_list[i] is not None:
    lat += [p.y if type(p)==shp.geometry.point.Point else None for p in lat_list[i]]
    lon += [p.x if type(p)==shp.geometry.point.Point else None for p in lon_list[i]]

fig.add_trace(go.Scattermapbox(
        lat=lat,
        lon=lon,
        mode='markers',
        marker=dict(
          color=["red" for x in lat],
          size=6,
        ), name="food shops"))


fig.update_layout(title="<b>Buildings across Milan<b>",
                  mapbox=dict(
                    style="open-street-map",
                    center=dict(lat=food_data.geometry.y.mean(), lon=food_data.geometry.x.mean()),
                    zoom=10
                    ),
                  title_pad_l=390,
                  height=700,
                  width=1100,
                  xaxis_title="time of the day",)

## Define the optimization model

In [ ]:
# define model constraints

    
# define the objective function

    
def __DefineAbstractModel(self):
    #-------------------------abastract model----------------------------
    model = AbstractModel()

    #---------------------------index sets-------------------------------
    model.I = Set()
    model.J = Set()

    #---------------------------parameters-------------------------------
    # define the distance matrix (d)
    model.d = Param(model.J, model.I, within = NonNegativeReals)

    #---------------------------variables--------------------------------
    # define the binary variables for the location decision (x)
    model.x = Var(model.J, within = Binary)

    # define the binary variables for the assignment decision (y)
    model.y = Var(model.I, model.J, within = Binary)

    #--------------------------constraints-------------------------------
    # define a constraint for each demand point to be covered by a single location
    model.completeSingleCoverage = Constraint(model.I, rule = self.__completeSingleCoverage)
        
    # define a constraint for the maximum number of locations 
    model.maximumLocations = Constraint(rule = self.__maximumLocations)
        
        
    # define a constraint for the maximal distance (L is an auxiliary variable)
    model.maximalDistance = Constraint(model.I, rule = self.__maximalDistance)
        
    # define a constraint for each demand point to be served by an open facility
    model.servedByOpenFacility = Constraint(model.I, model.J, rule = self.__servedByOpenFacility)

    #-----------------------objective function---------------------------
    model.maximalDistanceObj = Objective(rule = self.__maximalDistanceObj, sense = minimize)
        
    return model